In [1]:
from model import *
from data import *
import os
from tqdm import tqdm
import yaml
from utils import DotDict, adjust_learning_rate, accuracy
import torch
import traceback

In [2]:
losses = {'attn_lstm_vocab_10k': 10165.64429010069,
 'attn_lstm_vocab_1k': 15595.033411789249,
 'attn_lstm_vocab_50k': 6836.341175179966,
 'pointer_vocab_10k': 21672.131244335622,
 'pointer_vocab_1k': 43398.31753725069,
 'pointer_vocab_50k': 7057.6959175422135,
 'simple_lstm_vocab_10k': 20159.74126170967,
 'simple_lstm_vocab_1k': 109026.57322798869,
 'simple_lstm_vocab_50k': 13842.316331750844}


epoch = {'attn_lstm_vocab_10k': 8,
 'attn_lstm_vocab_1k': 5,
 'attn_lstm_vocab_50k': 1,
 'pointer_vocab_10k': 8,
 'pointer_vocab_1k': 5,
 'pointer_vocab_50k': 1,
 'simple_lstm_vocab_10k': 8,
 'simple_lstm_vocab_1k': 5,
 'simple_lstm_vocab_50k': 1}

acces = {'attn_lstm_vocab_10k': 0.6577360621117113,
 'attn_lstm_vocab_1k': 0.6494665779492861,
 'attn_lstm_vocab_50k': 0.6315070176565867,
 'pointer_vocab_10k': 0.6705065943604063,
 'pointer_vocab_1k': 0.6662379230942852,
 'pointer_vocab_50k': 0.634914800961635,
 'simple_lstm_vocab_10k': 0.657081022143478,
 'simple_lstm_vocab_1k': 0.6633081460420689,
 'simple_lstm_vocab_50k': 0.6168174382832627}

In [15]:
for config in os.listdir('configs'):
    if not config.endswith('yml'):
        continue
    config = os.path.join('configs', config)
    with open(config, 'r') as f:
        config = DotDict(yaml.safe_load(f))
    print('started', config.name)
    if config.name in acces:
        print('already calculated')
        continue
    checkpoint_folder = os.path.join('checkpoints', config.name)
    try:
        last_cpk = sorted(os.listdir(checkpoint_folder), key=lambda x: int(x[6:-4]), reverse=True)[0]
    except:
        traceback.print_exc()
        continue
    checkpoint_path = os.path.join(checkpoint_folder, last_cpk)
    
    device = config.train.device

    data_val = MainDataset(
        N_filename = config.data.N_filename,
        T_filename = config.data.T_filename,
        is_train=False,
        truncate_size=config.data.truncate_size
    )

    test_loader = torch.utils.data.DataLoader(
        data_val,
        batch_size=config.train.batch_size,
        shuffle=False,
        num_workers=config.train.num_workers,
        collate_fn=data_val.collate_fn
    )
    
    ignored_index = data_val.vocab_sizeT - 1
    unk_index = data_val.vocab_sizeT - 2
    
    model = MixtureAttention(
        hidden_size = config.model.hidden_size,
        vocab_sizeT = data_val.vocab_sizeT,
        vocab_sizeN = data_val.vocab_sizeN,
        attn_size = data_val.attn_size,
        embedding_sizeT = config.model.embedding_sizeT,
        embedding_sizeN = config.model.embedding_sizeN,
        num_layers = 1,
        dropout = config.model.dropout,
        label_smoothing = config.model.label_smoothing,
        pointer = config.model.pointer,
        attn = config.model.attn,
        device = device
    )
    cpk = torch.load(checkpoint_path)
    model.load_state_dict(cpk['model'])
    model = model.to(device)
    with torch.no_grad():
        model = model.eval()
        acc = 0.
        loss_eval = 0.
        for i, (n, t, p) in enumerate(tqdm(test_loader)):
            n, t, p = n.to(device), t.to(device), p.to(device)
            loss, ans = model(n, t, p)
            loss_eval += loss.item()
            acc += accuracy(ans.cpu().numpy().flatten(), t.cpu().numpy().flatten(), ignored_index, unk_index)
        acc /= len(test_loader)
        loss_eval /= len(test_loader)
        losses[config.name] = loss_eval
        acces[config.name] = acc
    print('loss:', losses[config.name], 'acc:', acces[config.name])
    torch.cuda.empty_cache()

started label_smoothing_pointer_10k
started pointer_vocab_10k
already calculated
started simple_lstm_vocab_10k
already calculated
started attn_lstm_vocab_50k
already calculated
started simple_lstm_vocab_1k
reading data from  ./pickle_data/PY_non_terminal_small.pickle


Traceback (most recent call last):
  File "<ipython-input-15-71a36957bcc8>", line 13, in <module>
    last_cpk = sorted(os.listdir(checkpoint_folder), key=lambda x: int(x[6:-4]), reverse=True)[0]
FileNotFoundError: [Errno 2] No such file or directory: 'checkpoints/label_smoothing_pointer_10k'


the vocab_sizeN is 330 (not including the eof)
the number of training data is 100000
the number of test data is 50000

reading data from  ./pickle_data/PY_terminal_1k_whole.pickle
the vocab_sizeT is 1000 (not including the unk and eof)
the attn_size is 50
the number of training data is 100000
the number of test data is 50000
Finish reading data and take 13.77



100%|██████████| 619/619 [04:29<00:00,  2.65it/s]


loss: 109026.57322798869 acc: 0.6633081460420689
started attn_lstm_vocab_10k
already calculated
started pointer_vocab_50k
already calculated
started pointer_vocab_1k
already calculated
started attn_lstm_vocab_1k
already calculated
started simple_lstm_vocab_50k
already calculated


In [3]:
import pandas as pd

/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [8]:
def make_table(dic, m = 1):
    data = pd.DataFrame(columns=['1k', '10k', '50k'], index=['simple_lstm', 'attn_lstm', 'pointer'])
    for item in dic:
        id_ = int([i for i, j in enumerate(item.split('_')) if j == 'vocab'][0])
        name = '_'.join(item.split('_')[:id_])
        vocab = item.split('_')[-1]
        data.loc[name, vocab] = dic[item]
    return data * m

In [5]:
acces

{'attn_lstm_vocab_10k': 0.6577360621117113,
 'attn_lstm_vocab_1k': 0.6494665779492861,
 'attn_lstm_vocab_50k': 0.6315070176565867,
 'pointer_vocab_10k': 0.6705065943604063,
 'pointer_vocab_1k': 0.6662379230942852,
 'pointer_vocab_50k': 0.634914800961635,
 'simple_lstm_vocab_10k': 0.657081022143478,
 'simple_lstm_vocab_1k': 0.6633081460420689,
 'simple_lstm_vocab_50k': 0.6168174382832627}

In [6]:
losses

{'attn_lstm_vocab_10k': 10165.64429010069,
 'attn_lstm_vocab_1k': 15595.033411789249,
 'attn_lstm_vocab_50k': 6836.341175179966,
 'pointer_vocab_10k': 21672.131244335622,
 'pointer_vocab_1k': 43398.31753725069,
 'pointer_vocab_50k': 7057.6959175422135,
 'simple_lstm_vocab_10k': 20159.74126170967,
 'simple_lstm_vocab_1k': 109026.57322798869,
 'simple_lstm_vocab_50k': 13842.316331750844}

In [9]:
make_table(acces, m = 100)

,1k,10k,50k
simple_lstm,66.3308,65.7081,61.6817
attn_lstm,64.9467,65.7736,63.1507
pointer,66.6238,67.0507,63.4915


In [10]:
make_table(losses)

,1k,10k,50k
simple_lstm,109027,20159.7,13842.3
attn_lstm,15595,10165.6,6836.34
pointer,43398.3,21672.1,7057.7


In [11]:
make_table(epoch)

,1k,10k,50k
simple_lstm,5,8,1
attn_lstm,5,8,1
pointer,5,8,1


### Eval from an example

In [12]:
from model import *
from data import *
import os
from tqdm import tqdm
import yaml
from utils import DotDict, adjust_learning_rate, accuracy
import torch
import traceback

In [13]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [16]:
config = 'configs/pointer_vocab_10k.yml'
with open(config, 'r') as f:
    config = DotDict(yaml.safe_load(f))
print('started', config.name)
checkpoint_folder = os.path.join('checkpoints', config.name)
last_cpk = sorted(os.listdir(checkpoint_folder), key=lambda x: int(x[6:-4]), reverse=True)[0]
checkpoint_path = os.path.join(checkpoint_folder, last_cpk)

device = 'cpu'

data_val = MainDataset(
    N_filename = config.data.N_filename,
    T_filename = config.data.T_filename,
    is_train=False,
    truncate_size=config.data.truncate_size
)

test_loader = torch.utils.data.DataLoader(
    data_val,
    batch_size=config.train.batch_size,
    shuffle=False,
    num_workers=config.train.num_workers,
    collate_fn=data_val.collate_fn
)

ignored_index = data_val.vocab_sizeT - 1
unk_index = data_val.vocab_sizeT - 2

model = MixtureAttention(
    hidden_size = config.model.hidden_size,
    vocab_sizeT = data_val.vocab_sizeT,
    vocab_sizeN = data_val.vocab_sizeN,
    attn_size = data_val.attn_size,
    embedding_sizeT = config.model.embedding_sizeT,
    embedding_sizeN = config.model.embedding_sizeN,
    num_layers = 1,
    dropout = config.model.dropout,
    label_smoothing = config.model.label_smoothing,
    pointer = config.model.pointer,
    attn = config.model.attn,
    device = device
)

started pointer_vocab_10k
reading data from  ./pickle_data/PY_non_terminal_small.pickle
the vocab_sizeN is 330 (not including the eof)
the number of training data is 100000
the number of test data is 50000

reading data from  ./pickle_data/PY_terminal_10k_whole.pickle
the vocab_sizeT is 10000 (not including the unk and eof)
the attn_size is 50
the number of training data is 100000
the number of test data is 50000
Finish reading data and take 13.11



100%|██████████| 50000/50000 [00:33<00:00, 1504.67it/s]


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [17]:
cpk = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(cpk['model'])
model = model.to(device)

In [24]:
train_dataN, test_dataN, vocab_sizeN, train_dataT, test_dataT, vocab_sizeT, attn_size, train_dataP, test_dataP = input_data(
    config.data.N_filename, config.data.T_filename
)

reading data from  ./pickle_data/PY_non_terminal_small.pickle
the vocab_sizeN is 330 (not including the eof)
the number of training data is 100000
the number of test data is 50000

reading data from  ./pickle_data/PY_terminal_10k_whole.pickle
the vocab_sizeT is 10000 (not including the unk and eof)
the attn_size is 50
the number of training data is 100000
the number of test data is 50000
Finish reading data and take 9.66



In [49]:
f = open('pickle_data/terminal_dict_10k_PY.pickle', 'rb')
t_dict = pickle.load(f)
f.close()

In [55]:
len(t_dict['terminal_dict'])

10000

In [56]:
t_reversed = {val: key for key, val in t_dict['terminal_dict'].items()}

In [74]:
def decode(arr):
    return [t_reversed[item.item()] if item.item() < 10000 else item.item() - 10000 for item in arr]

In [70]:
sample_n = torch.tensor(test_dataN[5:6])
sample_t = torch.tensor(test_dataT[5:6])
sample_p = torch.tensor(test_dataP[5:6])

In [71]:
loss, ans = model(sample_n, sample_t, sample_p)

In [75]:
decode(sample_t[0])

['<empty>',
 'enum',
 0,
 'component',
 'Component',
 'object',
 'field',
 0,
 '<empty>',
 8,
 '<empty>',
 '<empty>',
 0,
 '0',
 '<empty>',
 0,
 '1',
 '<empty>',
 0,
 '2',
 '<empty>',
 0,
 '3',
 '<empty>',
 0,
 '<empty>',
 18,
 '<empty>',
 '<empty>',
 18,
 '0',
 '<empty>',
 0,
 '1',
 '<empty>',
 0,
 '2',
 '<empty>',
 0,
 '3',
 '<empty>',
 0,
 '<empty>',
 'Component',
 '<empty>',
 '<empty>',
 'enabled',
 '<empty>',
 'field',
 0,
 'bool',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 'materials',
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 'bool',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 'bool',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<

In [76]:
decode(ans[0])

['<empty>',
 '<empty>',
 'Enum',
 0,
 0,
 '<empty>',
 'Object',
 '<empty>',
 '<empty>',
 'object',
 '<empty>',
 '<empty>',
 0,
 '<empty>',
 '<empty>',
 0,
 '1',
 '<empty>',
 0,
 '2',
 '<empty>',
 0,
 '3',
 '<empty>',
 0,
 '<empty>',
 9,
 '<empty>',
 '<empty>',
 15,
 '2',
 '<empty>',
 24,
 '1',
 '<empty>',
 0,
 '2',
 '<empty>',
 0,
 '3',
 '<empty>',
 0,
 '<empty>',
 9,
 '<empty>',
 '<empty>',
 15,
 '1',
 '<empty>',
 'mandatory',
 '<empty>',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<empty>',
 0,
 '<empty>',
 'field',
 0,
 '<e